In [86]:
import os
import sys
import numpy as np
from collections import defaultdict
import gmpy2
import copy
import logging

inputfilepath = 'C:\EMOLGUI\A_O_C\Day24.txt'
#inputfilepath = 'C:\EMOLGUI\A_O_C\Day24.sample.txt'

In [87]:
class Blizzard():
    def __init__(self,position,direction):
        self.position = position
        self.direction = direction

    def setLimits(self,minX, maxX, minY, maxY):
        self.minX = minX
        self.maxX = maxX
        self.minY = minY
        self.maxY = maxY

    def move(self):
        if self.direction == '^':
            newPosition = (self.position[0]-1,self.position[1])
            if newPosition[0] < self.minX:
                newPosition = (self.maxX,newPosition[1])
        if self.direction == 'v':
            newPosition = (self.position[0]+1,self.position[1])
            if newPosition[0] > self.maxX:
                newPosition = (self.minX,newPosition[1])
        if self.direction == '<':
            newPosition = (self.position[0],self.position[1]-1)
            if newPosition[1] < self.minY:
                newPosition = (newPosition[0],self.maxY)
        if self.direction == '>':            
            newPosition = (self.position[0],self.position[1]+1)
            if newPosition[1] > self.maxY:
                newPosition = (newPosition[0],self.minY)
        self.position = newPosition

    def __repr__(self):
        return f'Blizzard at position {self.position} with movement {self.direction}'


In [88]:
def isFree(position, blizzardList):
    for blizzard in blizzardList:
        if blizzard.position == position:
            return False
    return True

In [89]:
blizzardList = []
rowCount = 0
minX, minY, maxX, maxY = 10,10,0,0
with open(inputfilepath,'r') as f:
    for line in f:
        minX = min(minX, rowCount)
        maxX = max(maxX, rowCount)
        if not('<' in line or '>' in line or '^' in line or 'v' in line):
            rowCount += 1
            continue
        colCount = 0
        rowItems = list(line.rstrip())
        for item in rowItems:
            minY = min(minY, colCount)
            maxY = max(maxY, colCount)
            if item == '#':
                colCount += 1
                continue

            if item == '.':
                colCount += 1
                continue

            blizzardList.append(Blizzard((rowCount,colCount),item))
            colCount += 1

        rowCount += 1

minX, maxX, minY, maxY = minX+1, maxX-1, minY+1, maxY-1

for blizzard in blizzardList:
    blizzard.setLimits(minX, maxX, minY, maxY)
    
startingPosition = (minX-1,minY)
finalPosition = (maxX+1,maxY)

minX, maxX, minY, maxY, startingPosition, finalPosition

(1, 25, 1, 120, (0, 1), (26, 120))

In [90]:

def isValidPosition(position, finalPosition, minX, minY, maxX, maxY):
    if position == finalPosition:
        return True
    if position[0] >= minX and position[0] <= maxX and position[1] >= minY and position[1] <= maxY:
        return True
    return False

def moveAll(blizzardList):
    for blizzard in blizzardList:
        blizzard.move()

def explore(blizzardList, pathsSoFar, finalPosition, minX, minY, maxX, maxY):
    
    nextPaths = []
    nextPositionSet = set()

    for curPath in pathsSoFar:
        #print(f'   Checking path {curPath}')
        startingPosition = curPath[-1]

        nextPositions = []
        nextPositions.append(startingPosition)

        nextPosition = (startingPosition[0]-1,startingPosition[1])
        if isValidPosition(nextPosition, finalPosition, minX, minY, maxX, maxY):
            nextPositions.append(nextPosition)

        nextPosition = (startingPosition[0]+1,startingPosition[1])
        if isValidPosition(nextPosition, finalPosition, minX, minY, maxX, maxY):
            nextPositions.append(nextPosition)

        nextPosition = (startingPosition[0],startingPosition[1]-1)
        if isValidPosition(nextPosition, finalPosition, minX, minY, maxX, maxY):
            nextPositions.append(nextPosition)

        nextPosition = (startingPosition[0],startingPosition[1]+1)
        if isValidPosition(nextPosition, finalPosition, minX, minY, maxX, maxY):
            nextPositions.append(nextPosition)

        if finalPosition in nextPositions:
            print(f'Found Exit!!!')
            nextPath = curPath + [finalPosition]
            print(nextPath)
            return [nextPath], True

        nextValidPositions = []

        for nextPosition in nextPositions:
            if isFree(nextPosition,blizzardList):
                if not nextPosition in nextPositionSet:
                    nextPositionSet.add(nextPosition)                
                    nextValidPositions.append(nextPosition)
   
        for nextPosition in nextValidPositions:
            nextPaths.append(curPath + [nextPosition])    

    return nextPaths, False

In [84]:
newBlizzardList = copy.deepcopy(blizzardList)
bestPath = [[startingPosition]]
found = False
round = 1
while not found:
    moveAll(newBlizzardList)
    bestPath, found = explore(newBlizzardList, bestPath, finalPosition, minX, minY, maxX, maxY)
    print(f'Round number: {round}, pathlist {len(bestPath)}')
    round += 1

print(bestPath)

Round number: 1, pathlist 2
Round number: 2, pathlist 3
Round number: 3, pathlist 2
Round number: 4, pathlist 2
Round number: 5, pathlist 2
Round number: 6, pathlist 2
Round number: 7, pathlist 3
Round number: 8, pathlist 4
Round number: 9, pathlist 3
Round number: 10, pathlist 3
Round number: 11, pathlist 2
Round number: 12, pathlist 3
Round number: 13, pathlist 6
Round number: 14, pathlist 6
Round number: 15, pathlist 6
Round number: 16, pathlist 8
Round number: 17, pathlist 9
Found Exit!!!
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 1), (1, 2), (1, 3), (1, 3), (2, 3), (3, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6)]
Round number: 18, pathlist 1
[[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (1, 1), (1, 2), (1, 3), (1, 3), (2, 3), (3, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6)]]


#### QUESTION2

In [91]:
newBlizzardList = copy.deepcopy(blizzardList)
bestPath = [[startingPosition]]
found = False
round = 1
while not found:
    moveAll(newBlizzardList)
    bestPath, found = explore(newBlizzardList, bestPath, finalPosition, minX, minY, maxX, maxY)
    print(f'Round number: {round}, pathlist {len(bestPath)}')
    round += 1

print(bestPath)

# Trip back to the entrance
bestPath = [[finalPosition]]
newFinalPosition = startingPosition
found = False
round = 1
while not found:
    moveAll(newBlizzardList)
    bestPath, found = explore(newBlizzardList, bestPath, newFinalPosition, minX, minY, maxX, maxY)
    print(f'Round number: {round}, pathlist {len(bestPath)}')
    round += 1

print(bestPath)

# Trip back to the exit
bestPath = [[startingPosition]]
newFinalPosition = finalPosition
found = False
round = 1
while not found:
    moveAll(newBlizzardList)
    bestPath, found = explore(newBlizzardList, bestPath, newFinalPosition, minX, minY, maxX, maxY)
    print(f'Round number: {round}, pathlist {len(bestPath)}')
    round += 1

print(bestPath)


Round number: 1, pathlist 2
Round number: 2, pathlist 3
Round number: 3, pathlist 3
Round number: 4, pathlist 4
Round number: 5, pathlist 4
Round number: 6, pathlist 3
Round number: 7, pathlist 2
Round number: 8, pathlist 3
Round number: 9, pathlist 4
Round number: 10, pathlist 4
Round number: 11, pathlist 6
Round number: 12, pathlist 8
Round number: 13, pathlist 9
Round number: 14, pathlist 9
Round number: 15, pathlist 13
Round number: 16, pathlist 16
Round number: 17, pathlist 12
Round number: 18, pathlist 12
Round number: 19, pathlist 22
Round number: 20, pathlist 24
Round number: 21, pathlist 28
Round number: 22, pathlist 32
Round number: 23, pathlist 30
Round number: 24, pathlist 34
Round number: 25, pathlist 36
Round number: 26, pathlist 44
Round number: 27, pathlist 43
Round number: 28, pathlist 52
Round number: 29, pathlist 56
Round number: 30, pathlist 63
Round number: 31, pathlist 68
Round number: 32, pathlist 73
Round number: 33, pathlist 82
Round number: 34, pathlist 83
Rou